In [ ]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()

if "GPU" not in device_name:
    print("GPU device not found")
else:
    print(f'Found GPU at: {device_name}')

# Alternative way to check
print("GPU", "available (YESS!!!!)" if tf.config.list_physical_devices("GPU") else "not available :(")


2025-05-18 10:21:47.974317: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747563708.151921      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747563708.208320      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Found GPU at: /device:GPU:0
GPU available (YESS!!!!)


I0000 00:00:1747563720.820404      35 gpu_device.cc:2022] Created device /device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [ ]:
!pip install tensorflow-addons

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 15.9 MB/s eta 0:00:0000:01
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.4.2
    Uninstalling typeguard-4.4.2:
      Successfully uninstalled typeguard-4.4.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydata-profiling 4.16.1 requires typeguard<5,>=3, but you have typeguard 2.13.3 which is incompatible.
inflect 7.5.0 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.


In [ ]:
!pip install transformers

# **PART 2**

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.9.0.13 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cufft-

In [ ]:
import logging
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification
from datasets import load_dataset
from torch.optim import AdamW
import pandas as pd

# Suppress transformer warnings
logging.getLogger("transformers.modeling_utils").setLevel(logging.ERROR)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("l3cube-pune/tamil-bert")

from datasets import load_dataset

# Load the Tamil NER dataset
dataset = load_dataset("ai4bharat/indic_glue", "wiki-ner.ta")
train_data = dataset['train']
validation_data = dataset['validation']
test_data = dataset['test']

tokenizer_config.json:   0%|          | 0.00/450 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/6.41M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/49.5k [00:00<?, ?B/s]

...

validation-00000-of-00001.parquet:   0%|          | 0.00/292k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/259k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20466 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2586 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2611 [00:00<?, ? examples/s]

In [ ]:
# Preprocessing the dataset to align NER tags with tokens
label_list = ["O", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC"]

def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    previous_word_id = None
    for word_id in word_ids:
        if word_id is None:
            new_labels.append(-100)  # Ignore these tokens in the loss
        elif word_id != previous_word_id:
            new_labels.append(labels[word_id])  # Only label first token of the word
        else:
            # Continue with I- tags for inner tokens, otherwise ignore
            new_labels.append(labels[word_id] if label_list[labels[word_id]].startswith("I-") else -100)
        previous_word_id = word_id
    return new_labels


def preprocess_data(batch):
    tokenized_inputs = tokenizer(batch["tokens"], truncation=True, is_split_into_words=True, padding="max_length", max_length=128)
    all_word_ids = [tokenized_inputs.word_ids(i) for i in range(len(batch["tokens"]))]

    # Align labels with word ids for each example in the batch
    all_aligned_labels = [
        align_labels_with_tokens(labels, word_ids) for labels, word_ids in zip(batch["ner_tags"], all_word_ids)
    ]

    # Add aligned labels to tokenized inputs
    tokenized_inputs["labels"] = all_aligned_labels
    return tokenized_inputs


train_dataset = train_data.map(preprocess_data, batched=True)
validation_dataset = validation_data.map(preprocess_data, batched=True)
test_dataset = test_data.map(preprocess_data, batched=True)

Map:   0%|          | 0/20466 [00:00<?, ? examples/s]

Map:   0%|          | 0/2586 [00:00<?, ? examples/s]

Map:   0%|          | 0/2611 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

# Load the model with more capacity for multilingual tasks
# Load model
model = AutoModelForTokenClassification.from_pretrained("l3cube-pune/tamil-bert", num_labels=len(label_list))

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/951M [00:00<?, ?B/s]

In [ ]:
!pip install seqeval

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=7896ff3d161f1ff79c575bf685a6fa1d2cb13a1b4743c594d05aab498f38a143
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


In [ ]:
from seqeval.metrics import classification_report, f1_score, accuracy_score, precision_score, recall_score
import numpy as np

# Define compute_metrics function
def compute_metrics(pred):
    label_ids = pred.label_ids
    predictions = pred.predictions.argmax(-1)

    # Remove ignored index (special tokens)
    true_labels = [[label_list[label_id] for (label_id, pred_id) in zip(label_ids[i], predictions[i]) if label_id != -100]
                   for i in range(len(label_ids))]
    true_predictions = [[label_list[pred_id] for (label_id, pred_id) in zip(label_ids[i], predictions[i]) if label_id != -100]
                        for i in range(len(predictions))]

    # Calculate metrics using seqeval
    precision = precision_score(true_labels, true_predictions)
    recall = recall_score(true_labels, true_predictions)
    f1 = f1_score(true_labels, true_predictions)
    accuracy = accuracy_score(true_labels, true_predictions)

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [ ]:
# A-LLRD Optimizer
def get_optimizer(model, base_lr=2e-5, min_decay=0.8, max_decay=0.95):
    def compute_gradient_norms(model):
        norms = []
        for i, layer in enumerate(model.bert.encoder.layer):
            norm = 0
            for param in layer.parameters():
                if param.grad is not None:
                    norm += param.grad.norm().item() ** 2
            norms.append(np.sqrt(norm) if norm > 0 else 1e-8)  # Avoid division by zero
        return norms

    class ALLRDOptimizer(AdamW):
        def __init__(self, params, model, base_lr, min_decay, max_decay, betas=(0.9, 0.999), eps=1e-8, amsgrad=False, weight_decay=0.01, fused=False, capturable=False, differentiable=False, maximize=False, foreach=None):
            defaults = dict(
                lr=base_lr,
                betas=betas,
                eps=eps,
                amsgrad=amsgrad,
                weight_decay=weight_decay,
                fused=fused,
                capturable=capturable,
                differentiable=differentiable,
                maximize=maximize,
                foreach=foreach
            )
            super().__init__(params, **defaults)
            self.model = model
            self.base_lr = base_lr
            self.min_decay = min_decay
            self.max_decay = max_decay
            self.step_count = 0

        def step(self, closure=None):
            self.step_count += 1
            # Compute gradient norms
            norms = compute_gradient_norms(self.model)
            # Clip norms to prevent instability
            norms = [min(max(norm, 1e-8), 10.0) for norm in norms]
            # Softmax over negative norms to get decay factors
            norms = torch.tensor(norms, device=next(self.model.parameters()).device)
            decay_factors = torch.softmax(-norms, dim=0).cpu().numpy()
            # Scale decay factors to [min_decay, max_decay]
            decay_factors = self.min_decay + (self.max_decay - self.min_decay) * decay_factors / decay_factors.max()
            # Log decay factors for debugging
            if self.step_count % 100 == 0:
                print(f"Step {self.step_count}: Decay factors: {decay_factors.tolist()}")
            # Update learning rates
            param_groups = []
            for i, layer in enumerate(self.model.bert.encoder.layer):
                layer_lr = self.base_lr * decay_factors[i]
                param_groups.append({
                    "params": layer.parameters(),
                    "lr": layer_lr,
                    "betas": self.defaults["betas"],
                    "eps": self.defaults["eps"],
                    "amsgrad": self.defaults["amsgrad"],
                    "weight_decay": self.defaults["weight_decay"],
                    "fused": self.defaults["fused"],
                    "capturable": self.defaults["capturable"],
                    "differentiable": self.defaults["differentiable"],
                    "maximize": self.defaults["maximize"],
                    "foreach": self.defaults["foreach"]
                })
            param_groups.append({
                "params": self.model.classifier.parameters(),
                "lr": self.base_lr,
                "betas": self.defaults["betas"],
                "eps": self.defaults["eps"],
                "amsgrad": self.defaults["amsgrad"],
                "weight_decay": self.defaults["weight_decay"],
                "fused": self.defaults["fused"],
                "capturable": self.defaults["capturable"],
                "differentiable": self.defaults["differentiable"],
                "maximize": self.defaults["maximize"],
                "foreach": self.defaults["foreach"]
            })
            self.param_groups = param_groups
            super().step(closure)

    optimizer_grouped_parameters = [
        {
            "params": layer.parameters(),
            "lr": base_lr,
            "betas": (0.9, 0.999),
            "eps": 1e-8,
            "amsgrad": False,
            "weight_decay": 0.01,
            "fused": False,
            "capturable": False,
            "differentiable": False,
            "maximize": False,
            "foreach": None
        } for layer in model.bert.encoder.layer
    ] + [{
        "params": model.classifier.parameters(),
        "lr": base_lr,
        "betas": (0.9, 0.999),
        "eps": 1e-8,
        "amsgrad": False,
        "weight_decay": 0.01,
        "fused": False,
        "capturable": False,
        "differentiable": False,
        "maximize": False,
        "foreach": None
    }]
    return ALLRDOptimizer(optimizer_grouped_parameters, model, base_lr, min_decay, max_decay)

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="no",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    report_to="none",
)

# Initialize trainer
data_collator = DataCollatorForTokenClassification(tokenizer)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    optimizers=(get_optimizer(model), None)
)

# Train and evaluate
trainer.train()
metrics = trainer.evaluate(test_dataset)
metrics_df = pd.DataFrame([metrics])
print(metrics_df)

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.487800,0.431284,0.944161,0.874098,0.869281,0.871683
2,0.185700,0.228529,0.948705,0.882443,0.883122,0.882782
3,0.108900,0.171528,0.958390,0.897660,0.909394,0.903489
4,0.074300,0.214915,0.956517,0.904184,0.916699,0.910398
5,0.056700,0.217691,0.954843,0.886653,0.909266,0.897817
6,0.041500,0.233125,0.956955,0.905118,0.915673,0.910365
7,0.031700,0.258846,0.958310,0.908265,0.922466,0.915310
8,0.032500,0.275519,0.958669,0.913066,0.917980,0.915516
9,0.027600,0.299649,0.953408,0.895496,0.922466,0.908781
10,0.013800,0.323098,0.955122,0.898470,0.910675,0.904532


Step 100: Decay factors: [0.94980479409697, 0.9494550202565177, 0.9498562364482807, 0.95, 0.9499909695425707, 0.9498999669393817, 0.9496376155306159, 0.9494517500193445, 0.9487226251329173, 0.9483005529905544, 0.9477729240532231, 0.8862609557628953]
Step 200: Decay factors: [0.9361704569875762, 0.9325021624778391, 0.9382438493611214, 0.9419810125039428, 0.941155954177531, 0.9434328950225206, 0.9450196026476838, 0.9460467876921617, 0.9474314022241291, 0.9483983101034857, 0.95, 0.8925044531672379]
Step 300: Decay factors: [0.9342031701671301, 0.930416993953044, 0.9360350379494592, 0.9387927812348137, 0.943748911820122, 0.9472017295390311, 0.9486561660950649, 0.9490394082501806, 0.948785277377077, 0.95, 0.9492947856080602, 0.8933273725221456]
Step 400: Decay factors: [0.9325716332511265, 0.927266629692771, 0.9321821913721837, 0.932567150271381, 0.9385532113474326, 0.9409665404851845, 0.9452914066102701, 0.9470288145161702, 0.9485948239009319, 0.9496624534561171, 0.95, 0.8843568321900664]


   eval_loss  eval_accuracy  eval_precision  eval_recall   eval_f1  \
0    0.49688       0.937468        0.891628     0.890717  0.891172   

   eval_runtime  eval_samples_per_second  eval_steps_per_second  epoch  
0       11.3115                  230.828                 28.909   10.0  


In [ ]:
# Inspect the log history to understand the structure of logs
for log in trainer.state.log_history:
    print(log)


{'loss': 1.6292, 'grad_norm': 1.609941005706787, 'learning_rate': 1.9610003907776477e-05, 'epoch': 0.19538882375928096, 'step': 500}
{'loss': 1.1415, 'grad_norm': 1.0975897312164307, 'learning_rate': 1.9219226260257915e-05, 'epoch': 0.3907776475185619, 'step': 1000}
{'loss': 0.7913, 'grad_norm': 2.1315252780914307, 'learning_rate': 1.882844861273935e-05, 'epoch': 0.5861664712778429, 'step': 1500}
{'loss': 0.5972, 'grad_norm': 1.2684862613677979, 'learning_rate': 1.8437670965220792e-05, 'epoch': 0.7815552950371238, 'step': 2000}
{'loss': 0.4878, 'grad_norm': 3.5513241291046143, 'learning_rate': 1.804689331770223e-05, 'epoch': 0.9769441187964049, 'step': 2500}
{'eval_loss': 0.43128353357315063, 'eval_accuracy': 0.9441610203268235, 'eval_precision': 0.8740979381443299, 'eval_recall': 0.869281045751634, 'eval_f1': 0.8716828374991967, 'eval_runtime': 11.2302, 'eval_samples_per_second': 230.272, 'eval_steps_per_second': 28.851, 'epoch': 1.0, 'step': 2559}
{'loss': 0.3831, 'grad_norm': 4.6733

In [ ]:
# Log validation metrics per epoch
print("\nValidation Metrics per Epoch:")
validation_metrics = []
for log in trainer.state.log_history:
    if "eval_f1" in log:  # Validation metrics
        epoch_metrics = {
            "epoch": log.get("epoch", None),
            "eval_loss": log.get("eval_loss", None),
            "eval_accuracy": log.get("eval_accuracy", None),
            "eval_precision": log.get("eval_precision", None),
            "eval_recall": log.get("eval_recall", None),
            "eval_f1": log.get("eval_f1", None)
        }
        validation_metrics.append(epoch_metrics)
validation_df = pd.DataFrame(validation_metrics)
print(validation_df)

# Evaluate on test dataset (overall metrics)
print("\nTest Metrics:")
test_metrics = trainer.evaluate(test_dataset)
test_metrics_summary = {
    "test_accuracy": test_metrics["eval_accuracy"],
    "test_precision": test_metrics["eval_precision"],
    "test_recall": test_metrics["eval_recall"],
    "test_f1": test_metrics["eval_f1"]
}
test_metrics_df = pd.DataFrame([test_metrics_summary])
print(test_metrics_df)

# Compute per-class metrics manually
print("\nPer-Class Test Metrics:")
model.eval()
all_true_labels = []
all_pred_labels = []

# Run predictions on test dataset
for batch in trainer.get_test_dataloader(test_dataset):
    batch = {k: v.to(model.device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    predictions = outputs.logits.argmax(dim=-1).cpu().numpy()
    labels = batch["labels"].cpu().numpy()

    # Collect true and predicted labels, ignoring -100
    for i in range(len(labels)):
        true_labels = [label_list[l] for l, p in zip(labels[i], predictions[i]) if l != -100]
        pred_labels = [label_list[p] for l, p in zip(labels[i], predictions[i]) if l != -100]
        all_true_labels.extend(true_labels)
        all_pred_labels.extend(pred_labels)

# Compute classification report
class_report = classification_report([all_true_labels], [all_pred_labels], output_dict=True)
per_class_data = []
for label, metrics in class_report.items():
    if label in label_list:  # Only include actual NER labels
        per_class_data.append({
            "label": label,
            "precision": metrics["precision"],
            "recall": metrics["recall"],
            "f1-score": metrics["f1-score"],
            "support": metrics["support"]
        })
per_class_df = pd.DataFrame(per_class_data)
print(per_class_df)


Validation Metrics per Epoch:
    epoch  eval_loss  eval_accuracy  eval_precision  eval_recall   eval_f1
0     1.0   0.431284       0.944161        0.874098     0.869281  0.871683
1     2.0   0.228529       0.948705        0.882443     0.883122  0.882782
2     3.0   0.171528       0.958390        0.897660     0.909394  0.903489
3     4.0   0.214915       0.956517        0.904184     0.916699  0.910398
4     5.0   0.217691       0.954843        0.886653     0.909266  0.897817
5     6.0   0.233125       0.956955        0.905118     0.915673  0.910365
6     7.0   0.258846       0.958310        0.908265     0.922466  0.915310
7     8.0   0.275519       0.958669        0.913066     0.917980  0.915516
8     9.0   0.299649       0.953408        0.895496     0.922466  0.908781
9    10.0   0.323098       0.955122        0.898470     0.910675  0.904532
10   10.0   0.496880       0.937468        0.891628     0.890717  0.891172
11   10.0   0.496880       0.937468        0.891628     0.890717  0.8

In [ ]:
# Test model on example sentences
def test_model(sentences):
    model.eval()
    for sentence in sentences:
        tokenized_input = tokenizer(sentence, truncation=True, padding="max_length", max_length=128, return_tensors="pt")
        input_ids = tokenized_input["input_ids"].to(model.device)
        attention_mask = tokenized_input["attention_mask"].to(model.device)
        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)
        predictions = np.argmax(outputs.logits.cpu().numpy(), axis=2)
        tokens = tokenizer.convert_ids_to_tokens(input_ids[0].cpu().numpy())
        predicted_labels = [label_list[p] for p in predictions[0]]
        print(f"\nTesting sentence: {sentence}")
        for token, label in zip(tokens, predicted_labels):
            if token not in tokenizer.special_tokens_map.values() and token.strip():
                print(f"Word: '{token}', Predicted Label: {label}")

test_sentences = [
    "என் பெயர் வின்ஸ்டன் சர்ச்சில் மற்றும் நான் கோயம்புத்தூரில் வருகிறேன்.",
    "இது ஒரு புதிய தொழில்நுட்ப மேம்பாடு.",
    "தமிழ் மொழி தமிழ்நாட்டின் நிலையான மொழியாகும்.",
    "இத்தாவரம் சீனா, இந்தியா, ஜப்பான், பப்பாசியா, மைக்குரோனீசியா போன்ற நாடுகளில் காணப்படுகிறது.",
    "சைஃப் அலி கான், தீபிகா படுகோண், ஜான் ஆபிரகாம் (நடிகர்), ஜாக்குலின் பெர்னாண்டஸ், அனில் கபூர், அமீஷா பட்டேல், மற்றும் ரஜினிகாந்த் ஆகியோர்."
]
test_model(test_sentences)


Testing sentence: என் பெயர் வின்ஸ்டன் சர்ச்சில் மற்றும் நான் கோயம்புத்தூரில் வருகிறேன்.
Word: 'என்', Predicted Label: I-LOC
Word: 'பெயர்', Predicted Label: I-LOC
Word: 'வின்', Predicted Label: I-PER
Word: '##ஸ்டன்', Predicted Label: I-PER
Word: 'சர்ச்', Predicted Label: B-LOC
Word: '##சில்', Predicted Label: B-LOC
Word: 'மற்றும்', Predicted Label: I-LOC
Word: 'நான்', Predicted Label: I-LOC
Word: 'கோ', Predicted Label: O
Word: '##யம்', Predicted Label: I-LOC
Word: '##புத்', Predicted Label: I-LOC
Word: '##தூரில்', Predicted Label: I-LOC
Word: 'வருகிறேன்', Predicted Label: I-LOC
Word: '.', Predicted Label: I-LOC

Testing sentence: இது ஒரு புதிய தொழில்நுட்ப மேம்பாடு.
Word: 'இது', Predicted Label: I-LOC
Word: 'ஒரு', Predicted Label: I-LOC
Word: 'புதிய', Predicted Label: I-LOC
Word: 'தொழில்நுட்ப', Predicted Label: O
Word: 'மேம்பாடு', Predicted Label: B-ORG
Word: '.', Predicted Label: I-LOC

Testing sentence: தமிழ் மொழி தமிழ்நாட்டின் நிலையான மொழியாகும்.
Word: 'தமிழ்', Predicted Label: O
Wor